In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,56,2024-02-16,-4,114.490909,95.872727,0.541164,10.990909,29.432727,0.204764,119.514545,...,10.016667,25.075926,0.247741,120.216667,0,2,4,2,20,2024
1,55,2024-02-16,-3,112.870370,99.877778,0.535185,13.037037,25.325926,0.195259,112.455556,...,11.308929,23.548214,0.179411,121.726786,0,2,4,2,9,2024
2,56,2024-02-16,1,111.727273,97.029091,0.537291,12.410909,26.081818,0.227491,114.394545,...,11.537736,24.903774,0.182849,117.575472,0,2,4,2,22,2024
3,55,2024-02-16,-2,113.351852,97.005556,0.538056,10.570370,24.981481,0.178370,116.107407,...,11.454545,25.300000,0.187091,115.101818,0,2,4,2,3,2024
4,56,2024-02-16,2,117.600000,97.954545,0.570055,12.685455,24.434545,0.241455,119.561818,...,10.716364,22.247273,0.201509,118.925455,0,2,4,2,24,2024
5,54,2024-02-16,1,118.264151,97.177358,0.569472,11.088679,24.662264,0.220736,121.486792,...,10.512963,20.496296,0.208852,121.314815,0,2,4,2,13,2024
6,56,2024-02-16,6,120.672727,97.518182,0.572436,10.605455,24.420000,0.196618,122.780000,...,10.690909,24.361818,0.188182,115.529091,0,2,4,2,2,2024
7,55,2024-02-16,-1,113.333333,98.225926,0.529333,10.979630,24.409259,0.204704,114.335185,...,11.354545,25.581818,0.214545,118.790909,0,2,4,2,11,2024
8,55,2024-02-16,3,108.648148,97.564815,0.529389,11.842593,22.150000,0.169815,111.037037,...,13.012500,29.253571,0.211625,117.789286,0,2,4,2,5,2024
9,55,2024-02-16,-8,114.777778,102.555556,0.541407,11.838889,19.931481,0.177426,111.883333,...,10.692727,26.005455,0.179164,118.985455,0,2,4,2,30,2024


In [6]:
import joblib

In [10]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-16,NYK,PHI,1.952381,1.869565,0
1,2024-02-16,DET,IND,5.400000,1.166667,0
2,2024-02-16,ORL,CLE,3.600000,1.307692,0
3,2024-02-16,BRK,TOR,1.952381,1.869565,0
4,2024-02-16,PHO,DAL,2.100000,1.769231,0
5,2024-02-16,LAC,OKC,2.140000,1.740741,0
6,2024-02-16,BOS,CHI,1.359712,3.250000,1
7,2024-02-16,HOU,NOP,3.250000,1.359712,0
8,2024-02-16,CHO,UTA,4.400000,1.227273,0
9,2024-02-16,WAS,DEN,9.500000,1.068966,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-16,NYK,PHI,1.952381,1.869565,0
1,2024-02-16,DET,IND,5.400000,1.166667,0
2,2024-02-16,ORL,CLE,3.600000,1.307692,0
3,2024-02-16,BRK,TOR,1.952381,1.869565,0
4,2024-02-16,PHO,DAL,2.100000,1.769231,0
5,2024-02-16,LAC,OKC,2.140000,1.740741,0
6,2024-02-16,BOS,CHI,1.359712,3.250000,1
7,2024-02-16,HOU,NOP,3.250000,1.359712,0
8,2024-02-16,CHO,UTA,4.400000,1.227273,0
9,2024-02-16,WAS,DEN,9.500000,1.068966,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-16,NYK,PHI,1.952381,1.869565,0
1,2024-02-16,DET,IND,5.400000,1.166667,0
2,2024-02-16,ORL,CLE,3.600000,1.307692,0
3,2024-02-16,BRK,TOR,1.952381,1.869565,0
4,2024-02-16,PHO,DAL,2.100000,1.769231,0
5,2024-02-16,LAC,OKC,2.140000,1.740741,0
6,2024-02-16,BOS,CHI,1.359712,3.250000,1
7,2024-02-16,HOU,NOP,3.250000,1.359712,0
8,2024-02-16,CHO,UTA,4.400000,1.227273,0
9,2024-02-16,WAS,DEN,9.500000,1.068966,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-16,NYK,PHI,1.952381,1.869565,0
1,2024-02-16,DET,IND,5.400000,1.166667,0
2,2024-02-16,ORL,CLE,3.600000,1.307692,0
3,2024-02-16,BRK,TOR,1.952381,1.869565,0
4,2024-02-16,PHO,DAL,2.100000,1.769231,0
5,2024-02-16,LAC,OKC,2.140000,1.740741,0
6,2024-02-16,BOS,CHI,1.359712,3.250000,1
7,2024-02-16,HOU,NOP,3.250000,1.359712,0
8,2024-02-16,CHO,UTA,4.400000,1.227273,0
9,2024-02-16,WAS,DEN,9.500000,1.068966,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-16,NYK,PHI,1.952381,1.869565,0.0
1,2024-02-16,DET,IND,5.400000,1.166667,0.0
2,2024-02-16,ORL,CLE,3.600000,1.307692,0.0
3,2024-02-16,BRK,TOR,1.952381,1.869565,1.0
4,2024-02-16,PHO,DAL,2.100000,1.769231,0.0
5,2024-02-16,LAC,OKC,2.140000,1.740741,0.0
6,2024-02-16,BOS,CHI,1.359712,3.250000,1.0
7,2024-02-16,HOU,NOP,3.250000,1.359712,0.0
8,2024-02-16,CHO,UTA,4.400000,1.227273,0.0
9,2024-02-16,WAS,DEN,9.500000,1.068966,0.0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: NN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-16,NYK,PHI,1.952381,1.869565,0.0
1,2024-02-16,DET,IND,5.400000,1.166667,0.0
2,2024-02-16,ORL,CLE,3.600000,1.307692,0.0
3,2024-02-16,BRK,TOR,1.952381,1.869565,0.0
4,2024-02-16,PHO,DAL,2.100000,1.769231,1.0
5,2024-02-16,LAC,OKC,2.140000,1.740741,0.0
6,2024-02-16,BOS,CHI,1.359712,3.250000,1.0
7,2024-02-16,HOU,NOP,3.250000,1.359712,0.0
8,2024-02-16,CHO,UTA,4.400000,1.227273,0.0
9,2024-02-16,WAS,DEN,9.500000,1.068966,0.0
